In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time, datetime 

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean
pd.set_option('display.max_columns', None)
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [3]:
sql1 = '''
SELECT 
tt.order_number '订单号', tt.true_name '姓名', tt.user_mobile '手机号', tt.product_name '产品名称', tt.create_time '下单日期', tt.refund_date '预计还款日期', tt.reality_refund_date '实际还款日期', 
tt.money '应付金额', (tt.real_pay_money+tt.pay_discount_money) '实付金额',tt.pay_discount_money, tt.discount_all_money,tt.min_date, tt.max_date, tt.diff_date
FROM (
SELECT  t.order_number,t.true_name,t.user_mobile,t.product_name,t.create_time, t.refund_date,t.reality_refund_date,t.money,t.real_pay_money,t.min_date,
case when max_date>=DATE_ADD(CURRENT_DATE, INTERVAL -1 DAY) then CURRENT_DATE else t.max_date END max_date,
case when t.min_date=t.max_date then DATEDIFF(CURRENT_DATE, t.min_date) 
	        when max_date>CURRENT_DATE then DATEDIFF(CURRENT_DATE, t.min_date) 
			else DATEDIFF(t.max_date, t.min_date) END diff_date
,t.pay_discount_money, t.discount_all_money
FROM (
SELECT -- 查询总体订单和去重以及剔除商家 
om.order_number,tmu.true_name,om.user_mobile,tod.product_name,om.create_time,tos.refund_date,tos.reality_refund_date,tos.money,tos.real_pay_money,tos.pay_discount_money, tos.discount_all_money
,min(tos.refund_date)over(partition by om.order_number) min_date
,max(tos.refund_date)over(partition by om.order_number) max_date
FROM db_digua_business.t_postlease_receivables_monitoring   tprm 
LEFT JOIN db_digua_business.t_order om on tprm.order_id = om.id 
LEFT JOIN db_digua_business.t_merchant tmer on tmer.id = tprm.merchant_id
LEFT JOIN db_digua_business.t_order_stages tos on om.id=tos.order_id
-- LEFT JOIN db_digua_business.t_order_relet tor on om.id=tor.order_id
LEFT JOIN db_digua_business.t_order_details tod on tod.order_id=om.id 
LEFT JOIN db_digua_business.t_member_user tmu on tmu.mobile=om.user_mobile
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
WHERE om.`status` in (2, 3, 4) 
AND tmer.shop_type !=2  -- 剔除企业 1、自营店铺；2、入驻店铺；3、代运营店铺'
AND tprm.merchant_id  not in (15,99)  -- 剔除CPS渠道合作 汇客好租
AND tprm.model_number not like '%探路者%'
-- AND tos.refund_date = DATE_ADD(CURRENT_DATE,INTERVAL +1 DAY)
AND (tos.real_pay_money+tos.pay_discount_money)<(tos.money+tos.sesame_promo_money)
and tos.reality_refund_date is null
and pa.type!=4
and om.has_actual!=1
) t
) tt
WHERE diff_date<=0 and refund_date>=CURRENT_DATE and refund_date<=DATE_ADD(CURRENT_DATE, INTERVAL +2 DAY);
'''
df_order = query(sql1)
df = df_order.copy()

In [4]:
t_date = datetime.now().date()
t_date
df 

,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,实际还款日期,应付金额,实付金额,pay_discount_money,discount_all_money,min_date,max_date,diff_date
0,A2024082909382572,石维超,18583737402,iPhone15Pro 全新国行 顺丰包邮,2024-08-29 09:38:26,2025-08-01,None,556.06,0.0,0.0,0.0,2025-08-01,2025-07-31,-1
1,A2024082917121746,李敬茹,15030662531,iPhone15Pro 全新国行 顺丰包邮,2024-08-29 17:12:17,2025-08-01,None,556.06,0.0,0.0,0.0,2025-08-01,2025-07-31,-1
2,A20240829173805178,王磊,17330368884,iPhone15ProMax 首月1元 降价了 全新国行 顺丰包邮 低租金高通过,2024-08-29 17:38:06,2025-08-01,None,445.30,0.0,0.0,0.0,2025-08-01,2025-07-31,-1
3,A2024082919023714,崔舒婷,13283098626,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,2024-08-29 19:02:38,2025-08-01,None,312.15,0.0,0.0,0.0,2025-08-01,2025-07-31,-1
4,A2024083015171196,穆正兵,17785103450,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,2024-08-30 15:17:12,2025-08-02,None,449.57,0.0,0.0,0.0,2025-08-02,2025-07-31,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,A20250629194251394,段祉若,15187508742,iPhone 16 Pro Max 全新未激活 赠送壳膜 顺丰包邮,2025-06-29 19:42:52,2025-08-02,None,468.44,0.0,0.0,0.0,2025-08-02,2025-07-31,-2
1080,A20250629195654521,余青峰,16696331779,首月1元 iPhone16ProMax 全新未激活 非监管机 赠送壳膜 顺丰包邮,2025-06-29 19:56:55,2025-08-02,None,518.77,0.0,0.0,0.0,2025-08-02,2025-07-31,-2
1081,A2025062920011815,姜虹羽,19581338375,首月1元 iPhone 16 全新未激活 顺丰包邮 快速审核 【直播】,2025-06-29 20:01:19,2025-08-02,None,464.46,0.0,0.0,0.0,2025-08-02,2025-07-31,-2
1082,A20250629202026171,赵放,19881165671,首月1元 iPhone16ProMax 全新未激活 非监管机 赠送壳膜 顺丰包邮,2025-06-29 20:20:26,2025-08-02,None,518.77,0.0,0.0,0.0,2025-08-02,2025-07-31,-2


In [8]:
df[['手机号', '产品名称', '预计还款日期', '应付金额']].to_excel(f'F:/需求/仕程需求/未还款名单_{t_date}.xlsx', index=False)